In [1]:
!pip install groq

In [2]:
!pip install phidata python-dotenv requests

In [ ]:
import os
from dotenv import load_dotenv
import requests
from groq import Groq
from datetime import datetime

load_dotenv()
OPENWEATHER_API_KEY = os.getenv("OPENWEATHER_API_KEY")
GROQ_API_KEY = os.getenv("GROQ_API_KEY")



2a8784c0d6a4e2f4fa698e6a9a71cbb1
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


In [ ]:

class SpaceXLaunchAnalyzer:
    def __init__(self):
        self.groq_client = Groq(api_key=GROQ_API_KEY)
        
    def fetch_launch_data(self):
        """Get next SpaceX launch details with additional information"""
        try:
            launch_url = "https://api.spacexdata.com/v4/launches/next"
            launch_data = requests.get(launch_url, timeout=10).json()
            rocket_url = f"https://api.spacexdata.com/v4/rockets/{launch_data['rocket']}"
            pad_url = f"https://api.spacexdata.com/v4/launchpads/{launch_data['launchpad']}"
            rocket_data = requests.get(rocket_url, timeout=5).json()
            pad_data = requests.get(pad_url, timeout=5).json()
            launch_time = datetime.strptime(launch_data['date_utc'], '%Y-%m-%dT%H:%M:%S.%fZ')
            time_str = launch_time.strftime('%B %d, %Y at %H:%M UTC')
            
            return {
                'mission': launch_data['name'],
                'time': time_str,
                'rocket': rocket_data['name'],
                'payload': launch_data.get('payloads', ['Unknown'])[0],
                'launchpad': pad_data['name'],
                'location': f"{pad_data['latitude']:.4f}°N, {pad_data['longitude']:.4f}°E",
                'coordinates': (pad_data['latitude'], pad_data['longitude'])
            }
        except Exception as e:
            print(f"Error fetching launch data: {e}")
            return None

    def fetch_weather_data(self, lat, lon):
        """Get detailed weather forecast with 3-hour precision"""
        try:
            url = "https://api.openweathermap.org/data/2.5/forecast"
            params = {
                'lat': lat,
                'lon': lon,
                'units': 'metric',
                'appid': OPENWEATHER_API_KEY,
                'cnt': 8  
            }
            response = requests.get(url, params=params, timeout=10)
            response.raise_for_status()
            return response.json()
        except Exception as e:
            print(f"Error fetching weather data: {e}")
            return None

    def generate_weather_summary(self, weather_data):
        """Create detailed weather analysis with trends"""
        if not weather_data:
            return "Weather data unavailable"
        
        current = weather_data['list'][0]
        main = current['main']
        weather = current['weather'][0]
        wind = current['wind']
        temps = [period['main']['temp'] for period in weather_data['list'][:6]]
        temp_trend = "steady"
        if len(temps) > 1:
            change = temps[-1] - temps[0]
            if change > 2: temp_trend = "rising"
            elif change < -2: temp_trend = "falling"
        precipitation = False
        rain = current.get('rain', {}).get('3h', 0)
        snow = current.get('snow', {}).get('3h', 0)
        
        summary = (
            f"\n🌡️ Current Temperature: {main['temp']:.1f}°C (feels like {main['feels_like']:.1f}°C, {temp_trend})\n"
            f"🌬️ Wind: {wind['speed']} m/s from {self.deg_to_compass(wind.get('deg', 0))}\n"
            f"💧 Humidity: {main['humidity']}% | 🏔️ Pressure: {main['pressure']} hPa\n"
            f"☁️ Conditions: {weather['description'].capitalize()} (Cloud cover: {current['clouds']['all']}%)\n"
        )
        
        if rain > 0 or snow > 0:
            summary += f"🌧️ Precipitation: {rain}mm rain, {snow}mm snow in last 3 hours\n"
        summary += "\n📉 Next 24 Hours Forecast:\n"
        for period in weather_data['list'][:8]:  
            time = datetime.fromtimestamp(period['dt']).strftime('%H:%M')
            summary += (
                f"{time}: {period['weather'][0]['description']}, "
                f"{period['main']['temp']:.1f}°C, "
                f"Wind {period['wind']['speed']} m/s\n"
            )
        
        return summary

    def deg_to_compass(self, degrees):
        """Convert degrees to compass direction"""
        directions = ['N', 'NNE', 'NE', 'ENE', 'E', 'ESE', 'SE', 'SSE',
                    'S', 'SSW', 'SW', 'WSW', 'W', 'WNW', 'NW', 'NNW']
        return directions[round(degrees / 22.5) % 16]

    def assess_launch_risk(self, launch_info, weather_summary):
        """Generate detailed launch risk assessment"""
        prompt = f"""
        SpaceX Launch Risk Assessment:
        
        Mission: {launch_info['mission']}
        Scheduled: {launch_info['time']}
        Rocket: {launch_info['rocket']}
        Launch Site: {launch_info['launchpad']}
        Coordinates: {launch_info['location']}
        
        Current Weather Conditions:
        {weather_summary}
        
        Analyze these conditions against standard launch constraints:
        - Maximum sustained winds: 30 mph (13.4 m/s)
        - No lightning within 10 nautical miles
        - No precipitation at launch time
        - Cloud ceiling > 6,000 feet
        - Visibility > 4 nautical miles
        
        Provide:
        1. GO/NO-GO recommendation
        2. Primary weather concerns
        3. Probability of delay (0-100%)
        4. Suggested mitigation strategies
        """
        
        try:
            response = self.groq_client.chat.completions.create(
                model="llama3-70b-8192",
                messages=[{"role": "user", "content": prompt}],
                temperature=0.6
            )
            return response.choices[0].message.content
        except Exception as e:
            return f"Could not generate assessment: {str(e)}"

    def run_analysis(self):
        """Execute complete launch weather analysis"""
        print("\n🚀 Starting SpaceX Launch Weather Analysis")
        print("\n📡 Fetching launch information...")
        launch_info = self.fetch_launch_data()
        if not launch_info:
            return "Failed to retrieve launch data"
        
        print(f"\n🔭 Mission: {launch_info['mission']}")
        print(f"⏰ Scheduled: {launch_info['time']}")
        print(f"🛰️ Rocket: {launch_info['rocket']}")
        print(f"🏁 Launch Site: {launch_info['launchpad']}")
        print(f"📍 Location: {launch_info['location']}")
        print("\n🌤️ Retrieving weather forecast...")
        weather_data = self.fetch_weather_data(*launch_info['coordinates'])
        if not weather_data:
            return "Failed to retrieve weather data"
        weather_summary = self.generate_weather_summary(weather_data)
        risk_assessment = self.assess_launch_risk(launch_info, weather_summary)
        report = (
            f"\n=== SPACEX LAUNCH WEATHER ANALYSIS ===\n"
            f"Mission: {launch_info['mission']}\n"
            f"Launch Window: {launch_info['time']}\n"
            f"Location: {launch_info['launchpad']} ({launch_info['location']})\n"
            f"\n=== CURRENT WEATHER CONDITIONS ==="
            f"{weather_summary}"
            f"\n=== LAUNCH RISK ASSESSMENT ===\n"
            f"{risk_assessment}"
        )
        
        return report

In [6]:
if __name__ == "__main__":
    analyzer = SpaceXLaunchAnalyzer()
    report = analyzer.run_analysis()
    print(report)


🚀 Starting SpaceX Launch Weather Analysis

📡 Fetching launch information...

🔭 Mission: USSF-44
⏰ Scheduled: November 01, 2022 at 13:41 UTC
🛰️ Rocket: Falcon Heavy
🏁 Launch Site: KSC LC 39A
📍 Location: 28.6081°N, -80.6040°E

🌤️ Retrieving weather forecast...

=== SPACEX LAUNCH WEATHER ANALYSIS ===
Mission: USSF-44
Launch Window: November 01, 2022 at 13:41 UTC
Location: KSC LC 39A (28.6081°N, -80.6040°E)

=== CURRENT WEATHER CONDITIONS ===
🌡️ Current Temperature: 29.4°C (feels like 34.2°C, falling)
🌬️ Wind: 7.13 m/s from SE
💧 Humidity: 73% | 🏔️ Pressure: 1020 hPa
☁️ Conditions: Scattered clouds (Cloud cover: 41%)

📉 Next 24 Hours Forecast:
02:30: scattered clouds, 29.4°C, Wind 7.13 m/s
05:30: scattered clouds, 28.0°C, Wind 6.86 m/s
08:30: light rain, 26.1°C, Wind 4.81 m/s
11:30: light rain, 25.7°C, Wind 4.19 m/s
14:30: overcast clouds, 25.0°C, Wind 4.06 m/s
17:30: overcast clouds, 25.8°C, Wind 3.96 m/s
20:30: overcast clouds, 27.8°C, Wind 3.9 m/s
23:30: overcast clouds, 28.8°C, Wind 6.